# Job Posting Scraping

### Import Packages

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup 
import pymongo
import requests
import time
import json
import re

### General Fucntions

In [ ]:
def getdriver ():
    path=r'chromedriver.exe'
    #driver = webdriver.Chrome(executable_path='/Users/Sripriya Srinivasan/Downloads/chromedriver_win32/chromedriver')
    driver = webdriver.Chrome(executable_path=path)
    driver.implicitly_wait(10)
    driver.set_script_timeout(120)
    driver.set_page_load_timeout(30)
    return driver

In [ ]:
def writePage(fname,content):
    with open(fname, "w", encoding="utf-8") as file:
        file.write(str(content))

In [ ]:
def read_file (name):
    HTMLFile = open(name, "rb")
    htmlfiledata = HTMLFile.read()
    return BeautifulSoup(htmlfiledata, 'lxml')

In [ ]:
def connect_mongodb (db_name, collection_name):
    client = pymongo.MongoClient("mongodb://localhost:27017")
    db = client[db_name]
    collection = db[collection_name]
    return collection

In [ ]:
def loadWebsiteData (url):
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        page = requests.get(url,headers=headers)
        # Create a beautifulsoup object 
        return BeautifulSoup(page.text, 'lxml')
    except :
        print("Error connecting to website")

## LinkedIn

### Infinite Scroll and Save all results

In [ ]:
datascience_url = "https://www.linkedin.com/jobs/search?keywords=data%20scientist&location=california&geoId=&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0"
dataanalyst_url = "https://www.linkedin.com/jobs/search/?currentJobId=3489147354&geoId=102095887&keywords=Data%20Analyst&location=California%2C%20United%20States&refresh=true"
businessanalyst_url = ""

# Change assignment for scraping different roles
url = datascience_url

driver = getdriver()
driver.get(url)
time.sleep(2)  
scroll_pause_time = 1 
screen_height = driver.execute_script("return window.screen.height;")
i = 1
link_num = 1
while True:
    driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
    i += 1
    time.sleep(scroll_pause_time)
    scroll_height = driver.execute_script("return document.body.scrollHeight;") 
    try:
        button = driver.find_elements("xpath","//*[contains(text(), 'See more jobs')]")
        button[0].click()
    except:
        print("Button not found")
    if (screen_height) * i > scroll_height:
        break
writePage('linkedin_search_results',driver.page_source)
links = driver.find_elements("css selector","a.base-card__full-link.absolute")
files_list = []
for link in links:
    fname = 'ds_'+str(link_num)
    files_list.append(fname)
    writePage(fname,loadWebsiteData(link.get_attribute('href')))
    link_num = link_num + 1
    
print(len(links))
driver.close()

### Iterate through saved files and parse information

In [ ]:
job_posting_dict = {}
for file in files_list :
    job_soup = read_file(file)
    role = job_soup.select("h1")[0].text
    company = job_soup.select("a.topcard__org-name-link")[0].text.strip()
    location =  job_soup.select("span.topcard__flavor.topcard__flavor--bullet")[0].text.strip()
    info = job_soup.select("div.show-more-less-html__markup")[0].text.strip()

    fields = job_soup.select("h3.description__job-criteria-subheader")
    values = job_soup.select("span.description__job-criteria-text.description__job-criteria-text--criteria")
    extra_info = {}
    for i in range(0,len(fields)):
        extra_info[fields[i].text.strip()] = values[i].text.strip()
    posting = {'Job Role':role,'Company':company,'Location':location,'Job Description':info,'Additional Details':extra_info}
    job_posting_dict.append(posting)

In [ ]:
collection = connect_mongodb ('job_postings', 'jobs')
collection.insert_many(job_posting_dict)